# EUNIS habitat mapping in openEO - run of the inference part to generate habitat occurrence probabilities
In this notebook we generate the occurrence probability maps for each EUNIS habitat in the hierarchical CatBoost ML approach.

In [ ]:
import openeo
from openeo.extra.job_management import CsvJobDatabase
from eo_processing.utils import laea20km_id_to_extent, reproj_bbox_to_ll
from eo_processing.utils.helper import init_connection, location_visu
from eo_processing.utils.jobmanager import WeedJobManager
from eo_processing.utils.storage import get_gdrive_gdf, print_gdrive_overview, WEED_GDRIVE_Access
from eo_processing.openeo.processing import generate_master_feature_cube
from eo_processing.config import get_job_options, get_collection_options, get_standard_processing_options
import eo_processing.resources
import importlib.resources as importlib_resources
import geopandas as gpd
import pandas as pd
import os
from typing import Optional
import platform

### get the alpha-1 test sites from the GDrive (SK and CZ)

In [ ]:
# init the GDRIVE access
# provide the terrascope credentials to access the VITO vault
username = 'buchhornm'
gdrive = WEED_GDRIVE_Access(username=username, entry_point="1k27bitdRp41AtHq1xupyqwKaTLzrMUMu")

In [ ]:
# get an overview of the folder structure
print_gdrive_overview(gdrive)

In [ ]:
# load the test sides into a GeoPandas GeoDataFrame
path_SK = gdrive.root + "/" + 'SK_test-sites_alpha1_EPSG4326.gpkg'
path_CZ = gdrive.root + "/" + 'CZ_test-sites_alpha1_EPSG4326.gpkg'
# use helper function to load the GDrive file directly
gdf_sites = [get_gdrive_gdf(gdrive, path_file) for path_file in [path_SK, path_CZ]]
gdf_sites = gpd.GeoDataFrame(pd.concat(gdf_sites, ignore_index=True))

### specify space context for the openEO run
The space context is set by intersecting the test sites (AOIs) with the openEO EU 20x20km tiling grid. Since the tiling grid has a smaller file size, it was also stored in the resources of the eo_processing package as well as on the GDrive folder. <br>

In [ ]:
# get the LAEA-20km grid saved as EPSG:4326 vector
grid = importlib_resources.files(eo_processing.resources).joinpath('LAEA-20km.gpkg')
gdf_grid = gpd.read_file(os.path.normpath(grid))

In [ ]:
# spatial intersect and retrieve the tile_ID's of the LAEA-20km grid (convert AOI to EPSG:4326 if needed)
gdf_aoi = gdf_grid[gdf_grid.intersects(gdf_sites.union_all())]

In [ ]:
# visualization of the selected- tiles in the tiling grid which needs to be processed to cover the AOI
location_visu(gdf_aoi, zoom=True, region='EU', label=True)

### Integration of the MultiBackendJobManager


#### creating the jobs dataframe
The `MultiBackendJobManager` uses a jobs database to set-up, start and monitor all desired jobs. We are using Pandas/GeoPandas to create a DataFrame storing the job and all parameters which should adapted for each job.

In this example we can directly use the GeoDataFrame of the tiling grid since after the intersection with the AOI is contains only the remaining block extents to process.

In [ ]:
def create_job_dataframe(gdf: gpd.GeoDataFrame, year: int, file_name_base: str, target_crs: Optional[int] = None) -> gpd.GeoDataFrame:

    columns = ['name', 'target_epsg', 'file_prefix', 'start_date', 'end_date', 'geometry']
    dtypes = {'name': 'string', 'target_epsg': 'UInt16', 'file_prefix': 'string', 'start_date': 'string', 'end_date': 'string', 'geometry': 'geometry'}

    job_df = gdf.copy()

    # the time context is given by start and end date
    job_df['start_date'] = f'{year}-01-01'
    job_df['end_date'] = f'{year+1}-01-01'  # the end is always exclusive

    # adding the output file name pre-fix
    job_df['file_prefix'] = job_df.apply(lambda row: f'{file_name_base}_EUNIS-habitat-proba-cube_year{year}_{row['name']}', axis=1)

    # set the target epsg
    if target_crs is None:
        job_df['target_epsg'] = 3035
    else:
        raise NotImplementedError('target_crs is not yet implemented')

    return job_df.astype(dtypes)

In [ ]:
# we set here also the time context (processing year)
job_df = create_job_dataframe(gdf_aoi, 2021, 'alpha-1_test-sites')

In [ ]:
# show the jobs_database before conversion into a JobDatabaseInterface
job_df.head()

#### creating the job `start_job` function
The next step is to define a `start_job` function. This function will instruct the `MultiBackendJobManager` on how to initiate a new job on the selected backend. The `start_job` functionality should adhere to the following structure _start_job(row: pd.Series, connection: openeo.Connection, **kwargs)_.

In [ ]:
def inference(row: gpd.GeoSeries, connection: openeo.Connection, provider: str , **kwgs) -> openeo.BatchJob:
    """Start a new job using the specified row out of the jobs GeoDataFrame and connection."""

    # Get the variable parameters from the dataframe
    tileID = row["name"]
    start = row["start_date"]
    end = row["end_date"]
    file_name = row["file_prefix"]
    epsg = int(row["target_epsg"])

    # convert the row name into a openEO bbox dict giving the spatial extent of the job
    processing_extent = laea20km_id_to_extent(tileID)

    # define job_options, processing_options,  and collection_options
    job_options = get_job_options(provider=provider)
    collection_options = get_collection_options(provider=provider)
    processing_options = get_standard_processing_options(provider=provider, task='feature_generation')

    # adapt the epsg to the processing grid
    processing_options.update(target_crs = epsg)

    #create the progress graph for the feature cube
    # define the S1/S2 processed feature cube (Note: do not set spatial extent since we had it over in the end)
    data_cube = generate_master_feature_cube(connection,
                                             None,
                                             start,
                                             end,
                                             **collection_options,
                                             **processing_options)

    # now we merge in the NON ON-DEMAND processed features (DEM and WENR features)
    # load the DEM from a CDSE collection
    DEM = connection.load_collection(
        "COPERNICUS_30",
        bands=["DEM"])
    # reduce the temporal domain since copernicus_30 collection is "special" and feature only are one time stamp
    DEM = DEM.reduce_dimension(dimension='t', reducer=lambda x: x.last(ignore_nodata=True))
    # resample the cube to 10m and EPSG of corresponding 20x20km grid tile
    DEM = DEM.resample_spatial(projection=processing_options['target_crs'],
                               resolution=processing_options['resolution'],
                               method="bilinear")
    # merge into the S1/S2 data cube
    data_cube = data_cube.merge_cubes(DEM)

    # load the WERN features from public STAC
    WENR = connection.load_stac("https://stac.openeo.vito.be/collections/wenr_features")
    # resample the cube to 10m and EPSG of corresponding 20x20km grid tile
    WENR = WENR.resample_spatial(projection=processing_options['target_crs'],
                                 resolution=processing_options['resolution'],
                                 method="near")
    # drop the time dimension
    try:
        WENR = WENR.drop_dimension('t')
    except:
        # workaround if we still have the client issues with the time dimensions for STAC dataset with only one time stamp
        WENR.metadata = WENR.metadata.add_dimension("t", label=None, type="temporal")
        WENR = WENR.drop_dimension('t')
    # merge into the S1/S2 data cube
    data_cube = data_cube.merge_cubes(WENR)

    # filter spatial the whole cube
    data_cube = data_cube.filter_bbox(processing_extent)

    ####### fill here the inference workflow #####
    # ToDo: filled by Hans with the multiple catboost onnx inference workflow to create a datacube for the habitat probabilities from each applied model
    # format bands: Byte
    # nodata: 255
    # names bands: L#_habitat-#
    ######### end inference ######

    # create the job progress graph
    job = inference_cube.create_job(title=file_name,
                               description=f'generation of full feature data cube for tile {tileID}',
                               out_format='GTiff',
                               job_options=job_options,
                               filename_prefix=file_name,
                               )
    return job

#### set the root output folder

In [ ]:
# define the output folder (make sure you adapt this to your folder structure)
if platform.system() == 'Windows':
    out_root = os.path.normpath(r'\\netapp03.vgt.vito.be\habitat\slovakia\openEO_tests\alpha-1\5_inference-openEO')
else:
    out_root = os.path.normpath(r'/data/habitat/slovakia/openEO_tests/alpha-1/5_inference-openEO')
os.makedirs(out_root, exist_ok=True)

#### run the jobs
With our jobs database and job definition set up, we can now run the jobs using the `MultiBackendJobManager`. This involves defining a path to where we will store the job tracker which contains the job statuses and metadata.
<br> <br>
Note: If the specified job tracker path, points to an existing jobs database (either a .csv or .parquet file), that file will be used as the job tracker, and the input jobs database will be disregarded.

In [ ]:
# generate a unique name for the job tracker database (here: CSV file)
job_tracker = os.path.normpath(os.path.join(out_root, 'job_tracker_inference-openEO.csv'))

In [ ]:
# init of MultiBackendJobManager (A viz switch has been added to enable vizualization of status)
manager = WeedJobManager(root_dir=out_root, poll_sleep=30, viz=True)

In [ ]:
# create a connection to backend and add this active backend to the jobmanager
backend = 'cdse'  # also possible: cdse-stagging, terrascope, development, creodias, sentinelhub
connection = init_connection(backend)
manager.add_backend(backend, connection=connection, parallel_jobs=6)

In [ ]:
# due to a bug in the JobManager class we first have to create the CSVJobDatabase by ourself and write it to disk so that we get all custom columns!
job_db = CsvJobDatabase(path=job_tracker)
if job_db.exists():
    print(f"Resuming `run_jobs` from existing {job_db}")
else:
    df = manager._normalize_df(job_df)
    job_db.persist(df)

manager.run_jobs(start_job=featurecube_generation, job_db=job_db)

### validate that all needed feature data cubes are generated

In [ ]:
# load the job_tracker file into a pandas DataFrame
df_tracker = pd.read_csv(job_tracker)

In [ ]:
# Check if all values in column "status" have the value "finished"
all_finished = df_tracker['status'].eq('finished').all()
if all_finished:
    print("All jobs are finished.")
else:
    print("Some jobs are not finished. Reset the non finished status rows to 'not_started' flag and restart the manager.run_jobs() line manually.")

In [ ]:
# print the costs of the full job
print(f'The feature datacube generation {df_tracker.shape[0]} 20x20km tiles has cost overall {df_tracker.cost.sum()} credits ({df_tracker.cost.sum() * 0.01} EURO). ')

In [ ]:
# get the list of links to the GeoTif files
df_tracker['path_result'] = df_tracker['file_prefix'].apply(lambda x: os.path.normpath(os.path.join(out_root, f'{x}.tif')))
path_Results = df_tracker['path_result'].tolist()

In [ ]:
# Verify that all GeoTIFF files in path_Results exist
for file_path in path_Results:
    assert os.path.exists(file_path), f"File not found: {file_path}"